In [57]:
import MySQLdb
import pandas as pd

In [3]:
# used to find out which python jupyter uses.
!which python

/Users/chadleonard/miniconda2/envs/tutorial/bin/python


In [236]:
db = MySQLdb.connect(
                 host="localhost", 
                 user="root", 
                 passwd="mysqlpassword@2018", 
                 db="country_club" 
               )
cursor = db.cursor()
cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'facilities' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_fac = [col[0] for col in cursor.fetchall()]
print cols_fac

cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'bookings' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_book = [col[0] for col in cursor.fetchall()]
print cols_book

cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'members' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_mem = [col[0] for col in cursor.fetchall()]
print cols_mem

['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
['bookid', 'facid', 'memid', 'starttime', 'slots']
['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']


Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [148]:
sql = "select * from facilities where membercost > 0.0";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=cols_fac)
print df_fac['name'].values


['Tennis Court 1' 'Tennis Court 2' 'Massage Room 1' 'Massage Room 2'
 'Squash Court']


Q2: How many facilities do not charge a fee to members? 

In [146]:
sql = "select count(*) from facilities where membercost = 0.0";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=['col_count'])
df_fac['col_count'].values[0]


4

Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [189]:
cols = ['facid', 'name', 'membercost', 'monthlymaintenance' ]
sql = "select facid, name, membercost, monthlymaintenance \
       from ( \
                select facid \
                 , name \
                 , cast(membercost as decimal(4,2)) membercost \
                 , monthlymaintenance \
                 , (.20 * monthlymaintenance ) m \
                from facilities   \
            ) sub \
       where membercost < m and  membercost > 0";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
print df_fac[cols]


   facid            name membercost  monthlymaintenance
0      0  Tennis Court 1       5.00                 200
1      1  Tennis Court 2       5.00                 200
2      4  Massage Room 1       9.90                3000
3      5  Massage Room 2       9.90                3000
4      6    Squash Court       3.50                  80


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.

In [179]:
sql = "select * from facilities where facid in (1,5)";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=cols_fac)
print df_fac

   facid            name membercost guestcost  initialoutlay  \
0      1  Tennis Court 2        5.0      25.0           8000   
1      5  Massage Room 2        9.9      80.0           4000   

   monthlymaintenance  
0                 200  
1                3000  


Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question. 

In [258]:
sql = "select  name \
         ,  monthlymaintenance \
         ,  CASE \
               WHEN monthlymaintenance >  100 THEN 'expensive' \
               ELSE 'cheap' \
            END AS fac_type \
        from facilities ";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=['name', 'monthlymaintenance', 'fac_type'])
print df_fac

              name  monthlymaintenance   fac_type
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [199]:
sql = 'select firstname \
              , surname  \
          from members \
          where joindate = (select max(joindate) from members)'

cursor.execute(sql)
df_mem = pd.DataFrame([col for col in cursor.fetchall()], columns=['firstname', 'surname' ])
print df_mem

  firstname surname
0    Darren   Smith


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. 

In [224]:
# booking
ret_sql = "select distinct CONCAT(m.firstname, ' ',m.surname)  as membername \
               , f.name \
               from bookings b \
               join facilities f \
                 on b.facid = f.facid \
               join members m \
                 on b.memid = m.memid \
               where b.facid in (0,1) order by 1 ";

cursor.execute(cols_sql)

cols = ['membername', 'name' ]

print cols

cursor.execute(ret_sql)
df_bookings = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
df_bookings

['membername', 'name']


,membername,name
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 2
8,David Farrell,Tennis Court 1
9,David Jones,Tennis Court 1


Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [257]:
# booking
ret_sql = "select distinct f.name \
                  , CONCAT(m.firstname, ' ',m.surname)  as membername \
                  , case \
                      when b.memid <> 0 then f.membercost * b.slots \
                     else f.guestcost * b.slots \
                    end as cost \
                  , b.starttime \
               from bookings b \
                join facilities f \
                  on b.facid = f.facid \
                join members m \
                  on b.memid = m.memid \
                 where date(b.starttime) = '2012-09-14' \
                   and case \
                        when b.memid <> 0 then f.membercost * b.slots \
                          else f.guestcost * b.slots \
                        end > 30 \
                 order by 3 desc";


cols = ['Facility_Name', 'Member/Guest Name', 'Cost', 'starttime']

cursor.execute(ret_sql)
df_bookings = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
df_bookings[cols[:3]]

,Facility_Name,Member/Guest Name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Q9: This time, produce the same result as in Q8, but using a subquery.

In [261]:
ret_sql = "select * from \
         ( select distinct f.name \
                  , CONCAT(m.firstname, ' ',m.surname)  as membername \
                  ,  f.membercost * b.slots as cost \
                  , b.starttime \
               from bookings b \
                join facilities f \
                  on b.facid = f.facid \
                join members m \
                  on b.memid = m.memid \
                 where date(b.starttime) = '2012-09-14' and b.memid <> 0  \
                   and f.membercost * b.slots > 30 \
            union \
            select distinct f.name \
                  , CONCAT(m.firstname, ' ',m.surname)  as membername \
                  ,  f.guestcost * b.slots as cost \
                  , b.starttime \
               from bookings b \
                join facilities f \
                  on b.facid = f.facid \
                join members m \
                  on b.memid = m.memid \
                 where date(b.starttime) = '2012-09-14' and b.memid = 0  \
                   and f.guestcost * b.slots > 30 \
            ) sub \
            order by 3 desc";


cols = ['Facility_Name', 'Member/Guest Name', 'Cost', 'starttime']

cursor.execute(ret_sql)
df_bookings = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
df_bookings[cols[:3]]

,Facility_Name,Member/Guest Name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [281]:
ret_sql = "select * from \
           ( \
               select name, sum(revenue) total_revenue from \
                 ( select  f.name \
                          , f.membercost * b.slots as revenue \
                          , b.starttime \
                       from bookings b \
                        join facilities f \
                          on b.facid = f.facid \
                         where  b.memid <> 0  \
                    union \
                    select  f.name \
                          , f.guestcost * b.slots as revenue \
                          , b.starttime \
                       from bookings b \
                        join facilities f \
                          on b.facid = f.facid \
                         where b.memid = 0 \
                    ) sub \
                group by name \
             ) sub1 \
             where total_revenue < 1000 \
            order by 2";


cols = ['Facility_Name', 'Total_Revenue']

cursor.execute(ret_sql)
df_bookings = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
df_bookings[cols]

,Facility_Name,Total_Revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
